# Ayudantía 11:  I/O - Manejo de bytes
### Autores: Santiago Laguna (`@santilaguna`) y Benjamín Martínez (`@bimartinez`)


## Actividad 11, 2018-1

### Manejo de carpetas

Para poder resolver el primer problema, tenemos N carpetas que se encuentran dentro de otras M carpetas. En alguna de estas M X N carpetas se esconderán aleatoriamente dos archivos corruptos: marciano64.png y marcianozurdo.pep. Tu primera misión es encontrar automáticamente el path a cada uno de esos archivos corruptos, para poder leer esos archivos posteriormente

In [ ]:
import os


def encontrar_path(nombre_archivo):
    pass

### Rotar chunks

- Este algoritmo se aplica a una sub-secuencia de bytes, que denominaremos chunk. El tamaño de cada chunk se
definirá a la hora de juntar los archivos.
- El algoritmo consiste en que a cada chunk se le rota un byte a la izquierda, es decir, todos avanzan
una posición y el primero pasa a la última posición.

In [ ]:
# De la siguiente forma podemos rotar un chunk
# los bytearray tienen muchas funcionalidades en común con las listas
def rotar(chunk):
    nuevo = chunk[1:]
    nuevo.append(chunk[0])
    return nuevo

### Algoritmo base64


- Este algoritmo se lo debes aplicar a los bytes del archivo **marciano64.png**
- Leer los bytes del archivo marciano64.png.
- Transformar cada byte a su carácter ASCII correspondiente.
- Cada carácter obtenido se cambia por su valor base64 decimal. 

In [ ]:
import string


ARCHIVO_BASE64 = "marciano64.png" 
ARCHIVO_A_ROTAR =  "marcianozurdo.pep"
cwd = os.getcwd()  # current working directory

# todas las letras de base64, mayúsculas, minúsculas, "+" y "/".
mis_letras = string.ascii_uppercase + string.ascii_lowercase\
             + string.digits +  "+/"  

# para asignar a cada letra el número base64 que le corresponde
dict_letras = {mis_letras[i]: i for i in range(len(mis_letras))} 

In [ ]:
def algoritmo_base_64(nombre_archivo):
    with open(encontrar_path(nombre_archivo) + os.sep 
              + nombre_archivo, "rb") as file:
        letras = [chr(byte) for byte in file.read()]  # de ascii a letras
        base = [dict_letras[x] for x in letras]
        # ahora debemos convertir a binario de 6 bits, recordar el uso de zfill
        # luego los concatenamos, para esto debemos convertirlos a string.
        binarios = [str(bin(x)[2:].zfill(6)) for x in base]
        # también se puede de la siguiente forma
        # "{0:b}".format(x) > '100101'
        unidos = "".join(binarios)
        # dividimos en grupos de 8 bits.
        i = 0
        cadena = []
        while i < len(unidos):
            cadena.append(unidos[i: i + 8])
            i += 8
        # convertimos los binarios de 8 bits a sus equivalentes en decimales
        # guardaremos el entero con base 2 (binaria)
        return [int(binario, 2) for binario in cadena]

### Juntamos los archivos

Para crear este archivo de resultado, debes juntar chunks de ambos archivos en un bytearray. Estos se
agregarán al bytearray de manera intercalada partiendo por el archivo marcianozurdo.pep, de la siguiente
manera:
>- chunk 1: Aplicar algoritmo 2 al primer chunk del archivo marcianozurdo.pep, y agregar el resultado al bytearray.
>- chunk 2: Agregar el primer chunk del archivo marciano64.png, y agregar el resultado al bytearray.
>- ...

In [ ]:
def generador_size():
    """fibonacci generator"""
    a, b = 1, 1
    while True:
        yield a
        a, b = b, a + b

In [ ]:
def juntar(archivo_64, archivo_a_rotar):
    # abrimos los archivos respectivos
    base_64 = algoritmo_base_64(archivo_64)
    with open(encontrar_path(archivo_a_rotar) + os.sep 
              + archivo_a_rotar, "rb") as file:
        por_rotar = file.read()
    # creamos un bytearray que será el archivo de resultado que escribiremos
    array = bytearray()
    # inicializamos el generador fibonacci para el tamaño de los chunks
    fib = generador_size()
    inico = 0
    fin = len(base_64) + len(por_rotar)
    while inico < fin:
        # rotamos un chunk
        size = next(fib)
        chunk = rotar(bytearray(por_rotar[:size]))
        por_rotar = por_rotar[size:]
        # añadimos el chunk del archivo rotado
        array.extend(chunk)
        inico += size
        # añadimos el chunk del archivo en base_64
        size = next(fib)
        chunk = base_64[:size]
        base_64 = base_64[size:]
        array.extend(chunk)
        inico += size
    return array

In [ ]:
def escribir(text, archivo_resultado="resultado.png"):
    """escribimos un archivo binario"""
    with open(archivo_resultado, "wb") as file:
        file.write(text)

escribir(juntar(ARCHIVO_BASE64, ARCHIVO_A_ROTAR))